# Self-Driving Car Engineer Nanodegree


## Project: **Finding Lane Lines on the Road** 
***
In this project, you will use the tools you learned about in the lesson to identify lane lines on the road.  You can develop your pipeline on a series of individual images, and later apply the result to a video stream (really just a series of images). Check out the video clip "raw-lines-example.mp4" (also contained in this repository) to see what the output should look like after using the helper functions below. 

Once you have a result that looks roughly like "raw-lines-example.mp4", you'll need to get creative and try to average and/or extrapolate the line segments you've detected to map out the full extent of the lane lines.  You can see an example of the result you're going for in the video "P1_example.mp4".  Ultimately, you would like to draw just one line for the left side of the lane, and one for the right.

In addition to implementing code, there is a brief writeup to complete. The writeup should be completed in a separate file, which can be either a markdown file or a pdf document. There is a [write up template](https://github.com/udacity/CarND-LaneLines-P1/blob/master/writeup_template.md) that can be used to guide the writing process. Completing both the code in the Ipython notebook and the writeup template will cover all of the [rubric points](https://review.udacity.com/#!/rubrics/322/view) for this project.

---
Let's have a look at our first image called 'test_images/solidWhiteRight.jpg'.  Run the 2 cells below (hit Shift-Enter or the "play" button above) to display the image.

**Note: If, at any point, you encounter frozen display windows or other confounding issues, you can always start again with a clean slate by going to the "Kernel" menu above and selecting "Restart & Clear Output".**

---

**The tools you have are color selection, region of interest selection, grayscaling, Gaussian smoothing, Canny Edge Detection and Hough Tranform line detection.  You  are also free to explore and try other techniques that were not presented in the lesson.  Your goal is piece together a pipeline to detect the line segments in the image, then average/extrapolate them and draw them onto the image for display (as below).  Once you have a working pipeline, try it out on the video stream below.**

---

<figure>
 <img src="line-segments-example.jpg" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your output should look something like this (above) after detecting line segments using the helper functions below </p> 
 </figcaption>
</figure>
 <p></p> 
<figure>
 <img src="laneLines_thirdPass.jpg" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your goal is to connect/average/extrapolate line segments to get output like this</p> 
 </figcaption>
</figure>

## Import Packages

In [ ]:
#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2

%matplotlib inline

# additional modules used
from collections import deque
from ipywidgets import interact

## Read in an Image

In [ ]:
#reading in an image
image = mpimg.imread('test_images/extra.mp4-6.0.jpg')

#printing out some stats and plotting
print('This image is:', type(image), 'with dimesions:', image.shape)
plt.imshow(image)

## Make a Frame class

In [ ]:
class Frame:
    def load(filename):
        return Frame(mpimg.imread(filename))

    def weighted(fr1, w1, fr2, w2, c):
        return Frame(cv2.addWeighted(fr1.image, w1, fr2.image, w2, c))
    
    def zeros(shape):
        return Frame(np.ones(shape))
    
    def zeros_like(fr):
        return Frame(np.zeros_like(fr.image))

    def __init__(self, image):
        self.image = image

    def gray(self):
        return Frame(cv2.cvtColor(self.image, cv2.COLOR_RGB2GRAY))
    
    def sizexy(self):
        y, x, *_ = self.image.shape
        return (x, y)

    def x_at(self, xfrac):
        xdim, _ = self.sizexy()
        return np.array(xfrac) * xdim

    def y_at(self, yfrac):
        _, ydim = self.sizexy()
        return np.array(yfrac) * ydim

    def rgb(self):
        return Frame(cv2.cvtColor(self.image, cv2.COLOR_GRAY2RGB))

    def blur(self, kernel_size=5):
        return Frame(cv2.GaussianBlur(self.image, (kernel_size, kernel_size), 0))

    def canny(self, low=50, high_multiplier=3):
        return Frame(cv2.Canny(self.image, low, low * high_multiplier))
    
    def sobel(self, dx=1, dy=1, ksize=5, scale=1, delta=0):
        sobel64f = cv2.Sobel(self.image, cv2.CV_64F, dx, dy, ksize=ksize, scale=scale, delta=delta)
        abs_sobel64f = np.absolute(sobel64f)
        return Frame(np.uint8(abs_sobel64f))
    
    def laplacian(self, ksize=3):
        out = cv2.Laplacian(self.image, cv2.CV_64F, ksize=ksize)
        return Frame(out)

    def hist_eq(self):
        if self.is_monochrome():
            return Frame(cv2.equalizeHist(self.image))
        else:
            out = cv2.cvtColor(self.image, cv2.COLOR_RGB2HSV)
            out[:,:,2] = cv2.equalizeHist(out[:,:,2])
            return Frame(cv2.cvtColor(out, cv2.COLOR_HSV2RGB))

    def clahe_eq(self, clip_limit=2, grid_size=8):
        if self.is_monochrome():
            return Frame(cv2.createCLAHE(clipLimit=clip_limit, tileGridSize=(grid_size, grid_size)).apply(self.image))
        else:
            out = cv2.cvtColor(self.image, cv2.COLOR_RGB2HSV)
            out[:,:,2] = cv2.createCLAHE(clipLimit=clip_limit, tileGridSize=(grid_size, grid_size)).apply(out[:,:,2])
            return Frame(cv2.cvtColor(out, cv2.COLOR_HSV2RGB))

    def multiply_scalar(self, value):
        return Frame(cv2.addWeighted(self.image, value, 0, 0, 0))

    def add_scalar(self, value):
        return Frame(cv2.addWeighted(self.image, 1, 0, 0, value))

    def add_weighted(self, self_weight, frame, frame_weight, gamma):
        return Frame(cv2.addWeighted(self.image, self_weight, frame.image, frame_weight, gamma))

    def bitwise_or(self, frame):
        return Frame(cv2.bitwise_or(self.image, frame.image))

    def bitwise_and(self, frame):
        return Frame(cv2.bitwise_and(self.image, frame.image))

    def channels(self, channels):
        out = np.zeros_like(self.image)
        out[..., channels] = self.image[..., channels]
        return Frame(out)

    def channel(self, channel):
        return Frame(self.image[..., channel].copy())

    def hstack(self, frame):
        return Frame(np.hstack([self.image, frame.image]))

    def vstack(self, frame):
        return Frame(np.vstack([self.image, frame.image]))

    def fillpoly(self, vertices, color):
        return Frame(cv2.fillPoly(self.image.copy(), vertices, color))

    def draw_lines(self, lines, color, thickness):
        img = self.image.copy()
        for line in lines:
            x1, y1, x2, y2 = line
            cv2.line(img, (int(x1), int(y1)), (int(x2), int(y2)), color, thickness)
        return Frame(img)

    def draw_text(self, text, x, y, scale, color, thickness):
        return Frame(cv2.putText(self.image.copy(), text, (int(x), int(y)), cv2.FONT_HERSHEY_SIMPLEX, scale, color, thickness, cv2.LINE_AA))

    def hough_lines(self, rho, theta, threshold, min_line_length, max_line_gap):
        out = cv2.HoughLinesP(self.image, rho, theta, threshold, np.array([]), minLineLength=min_line_length, maxLineGap=max_line_gap)
        return out[:,0,:]

    def hsv_range_mask(self, hsv_ranges):
        if len(hsv_ranges) == 0:
            raise ValueError('At least one range is required')

        hsv = cv2.cvtColor(self.image, cv2.COLOR_RGB2HSV)

        range_lo, range_hi = hsv_ranges[0]
        mask = cv2.inRange(hsv, range_lo, range_hi)

        for hsv_range in hsv_ranges[1:]:
            range_lo, range_hi = hsv_range
            mask1 = cv2.inRange(hsv, range_lo, range_hi)
            mask = cv2.bitwise_or(mask, mask1)
            
        return Frame(mask)

    def is_monochrome(self):
        return len(self.image.shape) == 2

    def show(self, xdim=None):
        if xdim is not None:
            aspect = self.image.shape[0] / self.image.shape[1]
            plt.figure(figsize=(xdim, aspect * xdim))

        if self.is_monochrome():
            plt.imshow(self.image, cmap='gray', vmin=0, vmax=255)
        else:
            plt.imshow(self.image)

        plt.axis('off')
        plt.tight_layout()


class FrameGrid:
    def __init__(self, columns):
        self.columns = columns
        
    def show(self, frames, xdim=6):
        aspect = frames[0].image.shape[0] / frames[0].image.shape[1]
        ydim = aspect * xdim
        rows = len(frames)//self.columns + 1
        fig = plt.figure(figsize=(xdim * self.columns, ydim * rows))
        for i in range(len(frames)):
            if i < len(frames):
                fig.add_subplot(rows, self.columns, i + 1)
                frames[i].show()


class Lines:
    def from_x_intercepts(lines, xs):
        x1, x2 = xs
        y1, y2 = lines.y_intercept(xs)[0]
        return Lines([[x1, y1, x2, y2]])

    def from_y_intercepts(lines, ys):
        y1, y2 = ys
        x1, x2 = lines.x_intercept(ys)[0]
        return Lines([[x1, y1, x2, y2]])

    def __init__(self, lines):
        if type(lines) == np.ndarray:
            self.lines = lines
        else:
            self.lines = np.array(lines)
        
    def __len__(self):
        return self.lines.__len__()

    def __getitem__(self, i):
        return self.lines.__getitem__(i)

    def __iter__(self):
        return self.lines.__iter__()

    def empty(self):
        return len(self.lines) <= 0

    def centroid(self):
        x1, y1, x2, y2 = self.lines.T
        return np.array([(x1 + x2).T/2, (y1 + y2).T/2]).reshape([-1, 2])
    
    def mean(self):
        if len(self.lines) > 0:
            mean = self.lines.mean(axis=0)
            return Lines(mean.reshape(-1, 4))
        else:
            return Lines([])

    def m(self):
        x1, y1, x2, y2 = self.lines.T
        return ((y2 - y1) / (x2 - x1)).reshape((-1, 1))

    def b(self):
        x1, y1, x2, y2 = self.lines.T
        return ((x1 * y2 - x2 * y1) / (x1 - x2)).reshape((-1, 1))

    def y_intercept(self, x):
        return self.m() * x + self.b()

    def x_intercept(self, y):
        return (y - self.b()) / self.m()

    def theta(self):
        return np.arctan(self.m())

    def theta_degrees(self):
        return np.degrees(self.theta())

    def segment_by_theta(self, angle_pairs):
        out = []
        thetas = self.theta_degrees()
        for angle_pair in angle_pairs:
            lines = []
            for i, theta in enumerate(thetas):
                if angle_pair[0] < theta < angle_pair[1]:
                    lines.append(self.lines[i])
            out.append(lines)
            
        return [Lines(lines) for lines in out]


class LaneDetector:
    def __init__(self):
        self.roi = None
        self.params = {
            'roi': {
                'topy': 0.6,
                'bottomy': 1.0,
                'topw': 0.1,
                'bottomw': 0.9
            },
            'blur': {
                'kernel_size': 9,
            },
            'canny': {
                'low': 30,
                'high_multiplier': 3,
            },
            'hough': {
                'rho': 2,
                'theta': np.pi/180.0,
                'threshold': 35,
                'min_line_length': 20,
                'max_line_gap': 20
            }
        }

    def set_roi(self, frame, topy, bottomy, topw, bottomw):
        self.xdim, self.ydim = frame.sizexy()
        vertices = np.array([[
            (self.xdim * (0.5 - bottomw/2), self.ydim * bottomy),
            (self.xdim * (0.5 - topw/2),    self.ydim * topy),
            (self.xdim * (0.5 + topw/2),    self.ydim * topy),
            (self.xdim * (0.5 + bottomw/2), self.ydim * bottomy)
        ]], dtype=np.int32)

        self.roi = Frame.zeros_like(frame).fillpoly(vertices, [255])

    def detect(self, frame):
        self.gray = frame.gray()
        self.dark = self.gray.multiply_scalar(0.3)
        self.wymask = frame.hsv_range_mask([[(0, 0, 200), (255, 30, 255)], [(10, 90, 180), (40, 255, 255)]])
        self.gray_enhanced = self.dark.bitwise_or(self.wymask)
        self.gray_enhanced_blurred = self.gray_enhanced.blur(**self.params['blur'])
        self.edges = self.gray_enhanced_blurred.canny(**self.params['canny'])
        self.set_roi(self.edges, **self.params['roi'])
        self.masked_edges = self.edges.bitwise_and(self.roi)
        self.lines = Lines(self.masked_edges.hough_lines(**self.params['hough']))
        left, right = self.lines.segment_by_theta([(-38, -27), (27, 38)])

        return left.mean(), right.mean()


class LaneImageProcessor:
    def __init__(self, lane_detector, history_size=1, debug=False):
        self.lane_detector = lane_detector
        self.history_size = history_size
        self.left_history = deque()
        self.right_history = deque()
        self.debug = debug

    def smooth_and_draw(self, out, history, line, color):
        if not line.empty():
            x1, y1, x2, y2 = line[0]
            while len(history) < self.history_size + 1:
                history.appendleft([x1, y1, x2, y2])

            history.pop()

        if len(history) > 0:
            smooth = np.array(history).mean(axis=0)
            out = self.draw_lane(out, Lines([smooth]))
            
        return out

    def draw_lines(self, out, lines):
        if not lines.empty():
            out = out.draw_lines(lines, [0, 255, 0], 2)
            return out

    def draw_lane(self, out, mean_lane):
        if not mean_lane.empty():
            lane = Lines.from_y_intercepts(mean_lane, out.y_at([0.65, 1.0]))
            out = out.draw_lines(lane, [255, 0, 0], 10)

            centroid = lane.centroid()
            return out.draw_text("{:+.1f}".format(-lane.theta_degrees()[0][0]), centroid[0][0] - 70, centroid[0][1], 1, (255, 255, 255), 2)

    def process(self, frame):
        left, right = self.lane_detector.detect(frame)
        out = Frame.zeros_like(frame)
        out = self.smooth_and_draw(out, self.left_history, left, [255, 0, 0])
        out = self.smooth_and_draw(out, self.right_history, right, [255, 0, 0])
        
        if self.debug:
            im1 = self.lane_detector.gray_enhanced.rgb()
            im2 = self.lane_detector.masked_edges.rgb()
            im3 = self.draw_lines(self.lane_detector.masked_edges.rgb(), self.lane_detector.lines)
            im4 = Frame.weighted(frame, 1.0, out, 0.4, 0)
            row1 = im1.hstack(im2)
            row2 = im3.hstack(im4)
            return row1.vstack(row2)
        else:
            return Frame.weighted(frame, 1.0, out, 0.4, 0)

    def process_as_image(self, image):
        return self.process(Frame(image)).image

In [ ]:
# FrameGrid(2).show([
#     self.roi,
#     masked_edges,
# ], 8)

fr = Frame.load('test_images/extra.mp4-4.0.jpg')
# fr = Frame.load('test_images/extra.mp4-5.5.jpg')
# fr = Frame.load('test_images/extra.mp4-6.0.jpg')

detector = LaneDetector()
LaneImageProcessor(detector, history_size=1, debug=True).process(fr).show(15)

# fr.blur(7).gray().canny(50, 3).blurred(3).show()
# mask = fr.hsv_range_mask([[(0, 0, 200), (255, 30, 255)], [(10, 90, 180), (40, 255, 255)]])

# FrameGrid(2).show([
#     fr,
#     fr.blur(5).canny(),
#     mask.rgb().bitwise_and(fr),
#     mask.rgb().add_weighted(0.5, fr, 0.5, 0),
#     mask.rgb().bitwise_and(fr).blur().canny().blur(),
#     mask.rgb().bitwise_and(fr).blur().canny(),
# ], 8)

# FrameGrid(2).show([
#     mask.bitwise_and(fr.gray()).blur(11).canny(50, 3).blur(3),
#     fr
# ], 8)

## Explore the channels for useful information

In [ ]:
def copy_channels(img, channels):
    out = np.zeros_like(img)
    out[..., channels] = img[..., channels]
    return out

# fig = plt.figure(figsize=(16, 4))
# fig.add_subplot(1,4,1)
# plt.imshow(image)
# fig.add_subplot(1,4,2)
# plt.imshow(copy_channels(image, [0]))
# fig.add_subplot(1,4,3)
# plt.imshow(copy_channels(image, [1]))
# fig.add_subplot(1,4,4)
# plt.imshow(copy_channels(image, [2]))

i = image.copy()
h = cv2.cvtColor(i, cv2.COLOR_RGB2HSV)
h[..., [1]] = 255
i = cv2.cvtColor(h, cv2.COLOR_HSV2RGB)
plt.imshow(i)

In [ ]:
@interact(h1=(0, 255, 10), h2=(0, 255, 10), s1=(0, 255, 10), s2=(0, 255, 10), v1=(0, 255, 10), v2=(0, 255, 10))
def adjust_rgb(h1=0, h2=255, s1=0, s2=255, v1=220, v2=255):
    hsv = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
    mask = cv2.inRange(hsv, (h1, s1, v1), (h2, s2, v2))
    mask = cv2.GaussianBlur(mask, (5, 5), 0)
    plt.figure(figsize=(5, 5))
    plt.imshow(mask, cmap='gray')

In [ ]:
gr = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
fig = plt.figure(figsize=(5, 5))
# fig.add_subplot(311)
# plt.imshow(cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8)).apply(gr), cmap='gray')
# plt.imshow(gr, cmap='gray')
# plt.imshow(gr, cmap='gray')
# fig.add_subplot(312)
# plt.imshow(cv2.equalizeHist(gr), cmap='gray')
# fig.add_subplot(313)
# plt.imshow(cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8)).apply(gr), cmap='gray')
# plt.imshow(cv2.Sobel(image, cv2.CV_64F, 3, 3, ksize=5), cmap='gray')

sobelx64f = cv2.Sobel(gr,cv2.CV_64F,2,2,ksize=5)
abs_sobel64f = np.absolute(sobelx64f)
sobel_8u = np.uint8(abs_sobel64f)
plt.imshow(sobel_8u, cmap='gray')

## Ideas for Lane Detection Pipeline

**Some OpenCV functions (beyond those introduced in the lesson) that might be useful for this project are:**

`cv2.inRange()` for color selection  
`cv2.fillPoly()` for regions selection  
`cv2.line()` to draw lines on an image given endpoints  
`cv2.addWeighted()` to coadd / overlay two images
`cv2.cvtColor()` to grayscale or change color
`cv2.imwrite()` to output images to file  
`cv2.bitwise_and()` to apply a mask to an image

**Check out the OpenCV documentation to learn about these and discover even more awesome functionality!**

## Helper Functions

Below are some helper functions to help get you started. They should look familiar from the lesson!

In [ ]:
import math


def grayscale(img):
    """Applies the Grayscale transform
    This will return an image with only one color channel
    but NOTE: to see the returned image as grayscale
    (assuming your grayscaled image is called 'gray')
    you should call plt.imshow(gray, cmap='gray')"""
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Or use BGR2GRAY if you read an image with cv2.imread()
    # return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)


def canny(img, low_threshold, high_threshold):
    """Applies the Canny transform"""
    return cv2.Canny(img, low_threshold, high_threshold)


def equalize(img, cl=2.0, tilegrid=(8, 8)):
#     return cv2.createCLAHE(clipLimit=cl, tileGridSize=tilegrid).apply(img)
    return cv2.equalizeHist(img)


def white_mask(hsv):
    lower_white = np.array([0,0,220])
    upper_white = np.array([255,80,255])
    return cv2.inRange(hsv, lower_white, upper_white)


def yellow_mask(hsv):
    y_min, y_max = np.array([10, 50, 220]), np.array([50, 255, 255])
    return cv2.inRange(hsv, y_min, y_max)


def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)


def region_of_interest(img, vertices):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image


# def draw_lines_0(img, lines, color=[255, 0, 255], thickness=2):
#     """
#     NOTE: this is the function you might want to use as a starting point once you want to 
#     average/extrapolate the line segments you detect to map out the full
#     extent of the lane (going from the result shown in raw-lines-example.mp4
#     to that shown in P1_example.mp4).  
    
#     Think about things like separating line segments by their 
#     slope ((y2-y1)/(x2-x1)) to decide which segments are part of the left
#     line vs. the right line.  Then, you can average the position of each of 
#     the lines and extrapolate to the top and bottom of the lane.
    
#     This function draws `lines` with `color` and `thickness`.    
#     Lines are drawn on the image inplace (mutates the image).
#     If you want to make the lines semi-transparent, think about combining
#     this function with the weighted_img() function below
#     """
#     for line in lines:
#         for x1,y1,x2,y2 in line:
#             cv2.line(img, (x1, y1), (x2, y2), color, thickness)

left_lane_history = deque()
right_lane_history = deque()

def draw_lines(img, lines, color=[255, 0, 255], thickness=5):
    θmin = 27
    θmax = 38
    ymax = img.shape[0]

    mmin = np.tan(np.radians(θmin))
    mmax = np.tan(np.radians(θmax))

    left_lane_lines = []
    right_lane_lines = []

    def possible_left_lane_line(m):
        return mmin < m < mmax
    
    def possible_right_lane_line(m):
        return mmin < -m < mmax

    def line_to_mb(x1, y1, x2, y2):
        m = (y1 - y2) / (x1 - x2)
        b = (x1 * y2 - x2 * y1) / (x1 - x2)
        return (m, b)
    
    def frac_to_x(img, frac):
        _, xdim, _ = img.shape
        return frac * xdim
    
    def frac_to_y(img, frac):
        ydim, _, _ = img.shape
        return frac * ydim

    def y_intercept(x, m, b):
        return m * x + b

    def x_intercept(y, m, b):
        return (y - b) / m

    def smooth_and_draw(history, line, color):
        if line is not None:
            history_size = 10
            x1, y1, x2, y2 = line
            while len(history) < history_size + 1:
                history.appendleft([x1, y1, x2, y2])

            history.pop()

        if len(history) > 0:
            smooth = np.array(history).mean(axis=0, dtype=np.int)
            x3, y3, x4, y4 = smooth

            cv2.line(img, (int(x3), int(y3)), (int(x4), int(y4)), [255, 0, 0], 10)
            display_slope(x3, y3, x4, y4)

    def display_extrapolated_line(history, lines, color):
        if len(lines) > 0:
            line = np.array(lines).mean(axis=0, dtype=np.int)
            x1, y1, x2, y2 = line[0]
            m, b = line_to_mb(x1, y1, x2, y2)
            yfrac3, yfrac4 = 0.6, 1.0
            y3, y4 = frac_to_y(img, yfrac3), frac_to_y(img, yfrac4)
            x3, x4 = x_intercept(y3, m, b), x_intercept(y4, m, b)
            smooth_and_draw(history, (x3, y3, x4, y4), color)
#             cv2.line(img, (int(x3), int(y3)), (int(x4), int(y4)), color, thickness)
        else:
            smooth_and_draw(history, None, color)

#     def display_avg_line(lines, color):
#         if len(lines) > 0:
#             line = np.array(lines).mean(axis=0, dtype=np.int)
#             x1, y1, x2, y2 = line[0]
#             cv2.line(img, (int(x1), int(y1)), (int(x2), int(y2)), color, thickness//2)
#             display_slope(x1, y1, x2, y2)

    def display_slope(x1, y1, x2, y2):
        font = cv2.FONT_HERSHEY_SIMPLEX
        m = (y2 - y1) / (x2 - x1)
        cv2.putText(img, "{:+.2f}".format(np.degrees(-np.arctan(m))), (int(x1 + x2)//2 - 60, int(y1 + y2)//2), font, 1, (255,255,255), 2, cv2.LINE_AA)

    def display_lines(lines, color):
        if len(lines) > 0:
            for line in lines:
                for x1, y1, x2, y2 in line:
                    cv2.line(img, (int(x1), int(y1)), (int(x2), int(y2)), color, 2)
                    # display_slope(x1, y1, x2, y2)

    for line in lines:
        for x1, y1, x2, y2 in line:
            m = (y2 - y1) / (x2 - x1)
            if possible_left_lane_line(m):
                left_lane_lines.append(line)
            elif possible_right_lane_line(m):
                right_lane_lines.append(line)

    display_extrapolated_line(left_lane_history, left_lane_lines, [255, 255, 0])
    display_extrapolated_line(right_lane_history, right_lane_lines, [0, 255, 255])
    display_lines(left_lane_lines, [0, 255, 0])
    display_lines(right_lane_lines, [0, 255, 0])


def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    """
    `img` should be the output of a Canny transform.
        
    Returns an image with hough lines drawn.
    """
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    if lines is not None:
        draw_lines(line_img, lines, [255, 0, 0])
    return line_img

# Python 3 has support for cool math symbols.

def weighted_img(img, initial_img, α=1.0, β=0.8, λ=0.):
    """
    `img` is the output of the hough_lines(), An image with lines drawn on it.
    Should be a blank image (all black) with lines drawn on it.
    
    `initial_img` should be the image before any processing.
    
    The result image is computed as follows:
    
    initial_img * α + img * β + λ
    NOTE: initial_img and img must be the same shape!
    """
    return cv2.addWeighted(initial_img, α, img, β, λ)

## Test Images

Build your pipeline to work on the images in the directory "test_images"  
**You should make sure your pipeline works well on these images before you try the videos.**

In [ ]:
import os
os.listdir("test_images/")

## Build a Lane Finding Pipeline



Build the pipeline and run your solution on all test_images. Make copies into the test_images directory, and you can use the images in your writeup report.

Try tuning the various parameters, especially the low and high Canny thresholds as well as the Hough lines parameters.

In [ ]:
# TODO: Build your pipeline that will draw lane lines on the test_images
# then save them to the test_images directory.
import random

def trapezium(img_shape, yoffset=0.6, topwidth=0.1, bottomwidth=0.9):
    ydim, xdim, _ = img_shape
    return np.array([[
        (xdim * (0.5 - bottomwidth/2),ydim),
        (xdim * (0.5 - topwidth/2), ydim * yoffset),
        (xdim * (0.5 + topwidth/2), ydim * yoffset),
        (xdim * (0.5 + bottomwidth/2), ydim)
    ]], dtype=np.int32)


def mask_to_rgb(img):
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)


def darken(image, value):
    out = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)
    out = cv2.cvtColor(out, cv2.COLOR_RGB2HSV)
    out[:,:,2] = (out[:,:,2] * value).astype(int)
    out = cv2.cvtColor(out, cv2.COLOR_HSV2RGB)
    return cv2.cvtColor(out, cv2.COLOR_RGB2GRAY)


def find_lanes(image, kernel_size=5, canny_low=30, canny_high=90, hough_rho=1, hough_theta=np.pi/180, hough_thresh=30, hough_line_min_len=120, hough_line_max_gap=200, plot=False):
    """
    # hough_rho distance resolution in pixels of the Hough grid
    # hough_theta angular resolution in radians of the Hough grid
    # hough_thresh minimum number of votes (intersections in Hough grid cell)
    # hough_line_min_len minimum number of pixels making up a line
    # hough_line_max_gap maximum gap in pixels between connectable line segments
    """
    figsize=(20, 20)
    gray = grayscale(image)
    enhanced = equalize(gray)
    enhanced = darken(enhanced, 0.3)

    hsv = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
    ymask = yellow_mask(hsv)
    wmask = white_mask(hsv)
    mask = cv2.bitwise_or(ymask, wmask)
    enhanced = cv2.bitwise_or(enhanced, mask)

    blurred = gaussian_blur(enhanced, kernel_size)
    edges = canny(blurred, canny_low, canny_high)
#     return blurred

    masked_edges = region_of_interest(edges, trapezium(image.shape, yoffset=0.6, topwidth=0.08, bottomwidth=0.9))
#     return masked_edges
    if plot:
        plt.figure(figsize=figsize)
        plt.imshow(masked_edges, cmap='gray')

    lines_image = hough_lines(masked_edges, hough_rho, hough_theta, hough_thresh, hough_line_min_len, hough_line_max_gap)

    out = image.copy()
    detected = cv2.cvtColor(masked_edges, cv2.COLOR_GRAY2RGB)
    detected = weighted_img(detected, lines_image, α=1.0, β=0.7, λ=0.)
    out = weighted_img(detected, out, α=1.0, β=0.4, λ=0.)

#     out = image.copy()
#     out = weighted_img(lines_image, out, α=0.8, β=0.4, λ=0.)
    
    return out


def image_sample():
    images = os.listdir("test_images/")
    filename = "test_images/{}".format(random.choice(images))
    return filename

def find_all_lanes(directory):
    images = os.listdir(directory)
    for img in images:
        filename = "{}/{}".format(directory, img)
        image = mpimg.imread(filename)

        fig = plt.figure(figsize=(15, 15))
        fig.add_subplot(121)
        plt.imshow(image)
        
        fig.add_subplot(122)
        plt.imshow(find_lanes(image, **args))

In [ ]:
args3 = {
    'kernel_size': 7,
    'canny_low': 50,
    'canny_high': 150,
    'hough_rho': 1,
    'hough_theta': np.pi/180.0,
    'hough_thresh': 50,
    'hough_line_min_len': 80,
    'hough_line_max_gap': 120
}

args4 = {
    'kernel_size': 9,
    'canny_low': 30,
    'canny_high': 30 * 3,
    'hough_rho': 1,
    'hough_theta': np.pi/180.0,
    'hough_thresh': 35,
    'hough_line_min_len': 20,
    'hough_line_max_gap': 20
}

args = args4
directory = "test_images/"
images = os.listdir(directory)

@interact(img=images,kernel_size=(3,19,2), canny_low=(0,200,10), canny_ratio=[2,3],
          hough_rho=(1,20,1), hough_theta=(1,20,1), hough_thresh=(0,100,1),
          hough_line_min_len=(0,400,20), hough_line_max_gap=(0,400,20))
def detect(img, kernel_size=9, canny_low=30, canny_ratio=3,
           hough_rho=1, hough_theta=1, hough_thresh=35,
           hough_line_min_len=20, hough_line_max_gap=60):
    global left_lane_history
    global right_lane_history
    left_lane_history = deque()
    right_lane_history = deque()

    filename = "{}/{}".format(directory, img)
    image = mpimg.imread(filename)
    plt.figure(figsize=(10, 10))
    plt.imshow(find_lanes(image, kernel_size=kernel_size, canny_low=canny_low, canny_high=canny_low*canny_ratio,
                          hough_rho=hough_rho, hough_theta=hough_theta*np.pi/180., hough_thresh=hough_thresh,
                          hough_line_min_len=hough_line_min_len, hough_line_max_gap=hough_line_max_gap))

# find_all_lanes("test_images")

## Test on Videos

You know what's cooler than drawing lanes over images? Drawing lanes over video!

We can test our solution on two provided videos:

`solidWhiteRight.mp4`

`solidYellowLeft.mp4`

**Note: if you get an `import error` when you run the next cell, try changing your kernel (select the Kernel menu above --> Change Kernel).  Still have problems?  Try relaunching Jupyter Notebook from the terminal prompt. Also, check out [this forum post](https://carnd-forums.udacity.com/questions/22677062/answers/22677109) for more troubleshooting tips.**

**If you get an error that looks like this:**
```
NeedDownloadError: Need ffmpeg exe. 
You can download it by calling: 
imageio.plugins.ffmpeg.download()
```
**Follow the instructions in the error message and check out [this forum post](https://carnd-forums.udacity.com/display/CAR/questions/26218840/import-videofileclip-error) for more troubleshooting tips across operating systems.**

In [ ]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [ ]:
def process_image(image):
    # NOTE: The output you return should be a color image (3 channel) for processing video below
    # TODO: put your pipeline here,
    # you should return the final output (image where lines are drawn on lanes)

    return find_lanes(image, **args)

Let's try the one with the solid white lane on the right first ...

In [ ]:
left_lane_history = deque()
right_lane_history = deque()

white_output = 'out/white.mp4'
clip1 = VideoFileClip("solidWhiteRight.mp4")
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

Play the video inline, or if you prefer find the video in your filesystem (should be in the same directory) and play it in your video player of choice.

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))

## Improve the draw_lines() function

**At this point, if you were successful with making the pipeline and tuning parameters, you probably have the Hough line segments drawn onto the road, but what about identifying the full extent of the lane and marking it clearly as in the example video (P1_example.mp4)?  Think about defining a line to run the full length of the visible lane based on the line segments you identified with the Hough Transform. As mentioned previously, try to average and/or extrapolate the line segments you've detected to map out the full extent of the lane lines. You can see an example of the result you're going for in the video "P1_example.mp4".**

**Go back and modify your draw_lines function accordingly and try re-running your pipeline. The new output should draw a single, solid line over the left lane line and a single, solid line over the right lane line. The lines should start from the bottom of the image and extend out to the top of the region of interest.**

Now for the one with the solid yellow lane on the left. This one's more tricky!

In [ ]:
left_lane_history = deque()
right_lane_history = deque()

yellow_output = 'out/yellow.mp4'
clip2 = VideoFileClip('solidYellowLeft.mp4')
yellow_clip = clip2.fl_image(process_image)
%time yellow_clip.write_videofile(yellow_output, audio=False)

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(yellow_output))

## Writeup and Submission

If you're satisfied with your video outputs, it's time to make the report writeup in a pdf or markdown file. Once you have this Ipython notebook ready along with the writeup, it's time to submit for review! Here is a [link](https://github.com/udacity/CarND-LaneLines-P1/blob/master/writeup_template.md) to the writeup template file.


## Optional Challenge

Try your lane finding pipeline on the video below.  Does it still work?  Can you figure out a way to make it more robust?  If you're up for the challenge, modify your pipeline so it works with this video and submit it along with the rest of your project!

In [ ]:
left_lane_history = deque()
right_lane_history = deque()

challenge_output = 'out/extra.mp4'
clip2 = VideoFileClip('challenge.mp4')
# clip2 = clip2.resize((960, 540))
processor = LaneImageProcessor(LaneDetector(), history_size=10, debug=False)
challenge_clip = clip2.fl_image(processor.process_as_image)
%time challenge_clip.write_videofile(challenge_output, audio=False)

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(challenge_output))

In [ ]:
def extract_frames():
    challenge_output = 'out/extra.mp4'
    clip2 = VideoFileClip('challenge.mp4')

    for t in np.arange(0, 10, 0.5):
        clip2 = clip2.resize((960, 540))
        clip2.save_frame('in/{}-{}.jpg'.format(challenge_output.split('/')[-1], t), t)

# extract_frames()